# Réseaux convolutionnels : CNN

## Initialisations

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import model_selection

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

from sklearn import datasets

In [ ]:
from keras.datasets import mnist

from keras.models import Sequential, load_model

from keras.layers import Dense, Dropout, Flatten

from keras.layers.convolutional import Conv2D, MaxPooling2D

from keras.utils.np_utils import to_categorical

In [ ]:
# Permet d'éviter les erreurs mémoires pour le GPU
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

## Lecture des images

Sur kaggle, dataset "Chest X-Ray Images (Pneumonia)"
https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia

Pour une installation locale :
pip install opencv-python


In [ ]:
import cv2
import os
import glob

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

On parcourt le répertoire des images de poumons normaux (sans pneumonie, cible *y* à 0)

In [ ]:
img_dir = "../input/cat-and-dog/training_set/training_set/dogs/" 
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
X=[]
y=[]
for f1 in files:
    img = cv2.imread(f1)
    img = cv2.resize(img, (150,150))
    X.append(np.array(img))
    y.append(0)

In [ ]:
plt.imshow(X[0])
plt.title(y[0])

In [ ]:
X[0].shape

On parcourt le répertoire des images indiquant une pneumonie (cible *y* à 1)

In [ ]:
img_dir = "../input/cat-and-dog/training_set/training_set/cats/"
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
for f1 in files:
    img = cv2.imread(f1)
    img = cv2.resize(img, (150,150))
    X.append(np.array(img))
    y.append(1)

In [ ]:
plt.imshow(X[len(X)-1])
plt.title(y[len(X)-1])

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
# Normalisation entre 0 et 1
X = X.astype('float32') / 255

## Keras

In [ ]:
from keras.models import Sequential, load_model

from keras.layers import Dense, Dropout, Activation, Flatten

from keras.layers.convolutional import Conv2D, MaxPooling2D

from keras.utils.np_utils import to_categorical

In [ ]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [ ]:
from sklearn.model_selection import train_test_split
y = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)


In [ ]:
print(y_test[0])

### Dense

In [ ]:
num_classes = 2
# Réseau dense simple
model = Sequential()
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compilation du modèle
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
train = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=200, verbose=1)
model.summary()

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Score : %.2f%%" % (scores[1]*100))

### CNN

In [ ]:
# Réseau complexe
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

# Compilation du modèle
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
train = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=200, verbose=1)
model.summary()

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Score : %.2f%%" % (scores[1]*100))

y_pred = model.predict(X_test)

cm = confusion_matrix(y_test.argmax(axis = 1), y_pred.argmax(axis = 1))
print(cm)